In [22]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

# Group By

In [57]:
df = pd.DataFrame({
    "key1" : ["a", "a", None, "b", "b", "a", None],
    "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"), 
    "data1" : np.random.standard_normal(7), 
    "data2" : np.random.standard_normal(7)
})

df

,key1,key2,data1,data2
0,a,1,-0.176886,-0.948223
1,a,2,-1.182404,-1.136509
2,None,1,-0.027517,1.098178
3,b,2,-0.249308,0.003630
4,b,1,-1.609377,0.556054
5,a,<NA>,0.345657,-1.152459
6,None,1,-0.483572,-1.680686


In [24]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     1.810922  2.269297
     2    -0.530958  0.680683
b    1    -0.962111  0.765542
     2    -0.062327 -0.847670

In [25]:
df.groupby(["key1", "key2"])['data1'].mean()

key1  key2
a     1       1.810922
      2      -0.530958
b     1      -0.962111
      2      -0.062327
Name: data1, dtype: float64

In [26]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)
    print('--------')

a
  key1  key2     data1     data2
0    a     1  1.810922  2.269297
1    a     2 -0.530958  0.680683
5    a  <NA>  1.006755  1.449285
--------
b
  key1  key2     data1     data2
3    b     2 -0.062327 -0.847670
4    b     1 -0.962111  0.765542
--------


# Group by Filter

In [ ]:
# elections.groupby("Year").filter(lambda sf: sf["%"].max() < 45).head(9)

# Merge

In [27]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [28]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [29]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [30]:
pd.merge(df1, df2, on="key", how='inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [31]:
pd.merge(df1, df2, on="key", how='left')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


# Concat

Does not care about values unlike merge

In [32]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["b", "d", "e"], dtype="Int64")

In [33]:
s1

a    0
b    1
dtype: Int64

In [34]:
s2

b    2
d    3
e    4
dtype: Int64

In [56]:
pd.concat([s1, s2], axis=0)

a    0
b    1
b    2
d    3
e    4
dtype: Int64

In [36]:
pd.concat([s1, s2], axis=1)

,0,1
a,0,<NA>
b,1,2
d,<NA>,3
e,<NA>,4


# Pivot Table

Check out [Jake VDP](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html)

In [37]:
import seaborn as sns
titanic = sns.load_dataset('titanic')

## GroupBy Equivalent

In [38]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [39]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [41]:
titanic.groupby(['sex', 'class'], observed=False)['survived'].aggregate('mean')

sex     class 
female  First     0.968085
        Second    0.921053
        Third     0.500000
male    First     0.368852
        Second    0.157407
        Third     0.135447
Name: survived, dtype: float64

In [40]:
titanic.groupby(['sex', 'class'], observed=False)['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Pivot Table

In [42]:
titanic.pivot_table('survived', index='sex', columns='class', observed=False)

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Multi-level Pivot

In [43]:
age = pd.cut(titanic['age'], [0, 18, 80])

titanic.pivot_table(
    values='survived', 
    index=['sex', age], 
    columns='class', 
    observed=False
)

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [44]:
fare = pd.qcut(titanic['fare'], 2)

titanic.pivot_table(
    'survived', 
    index=['sex', age], 
    columns=[fare, 'class'], 
    observed=False
)

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

# Melt

In [45]:
df = pd.DataFrame({"Name": ["Tom", "Mike", "Tiffany", "Varada", "Joel"],
                   "2018": [1, 3, 4, 5, 3],
                   "2019": [2, 4, 3, 2, 1],
                   "2020": [5, 2, 4, 4, 3]})
df

,Name,2018,2019,2020
0,Tom,1,2,5
1,Mike,3,4,2
2,Tiffany,4,3,4
3,Varada,5,2,4
4,Joel,3,1,3


In [46]:
df_melt = df.melt(id_vars="Name",
                  var_name="Year",
                  value_name="Courses")
df_melt

,Name,Year,Courses
0,Tom,2018,1
1,Mike,2018,3
2,Tiffany,2018,4
3,Varada,2018,5
4,Joel,2018,3
5,Tom,2019,2
6,Mike,2019,4
7,Tiffany,2019,3
8,Varada,2019,2
9,Joel,2019,1


Let's omit a year!

In [47]:
df.melt(id_vars="Name",
        value_vars=["2019", "2020"],
        var_name="Year",
        value_name="Courses")

,Name,Year,Courses
0,Tom,2019,2
1,Mike,2019,4
2,Tiffany,2019,3
3,Varada,2019,2
4,Joel,2019,1
5,Tom,2020,5
6,Mike,2020,2
7,Tiffany,2020,4
8,Varada,2020,4
9,Joel,2020,3


Let's try pivot again here! 

In [49]:
df_melt

,Name,Year,Courses
0,Tom,2018,1
1,Mike,2018,3
2,Tiffany,2018,4
3,Varada,2018,5
4,Joel,2018,3
5,Tom,2019,2
6,Mike,2019,4
7,Tiffany,2019,3
8,Varada,2019,2
9,Joel,2019,1


# Pivot

In [48]:
df_pivot = df_melt.pivot(index="Name",
                         columns="Year",
                         values="Courses")
df_pivot

Year,2018,2019,2020
Name,,,
Joel,3,1,3
Mike,3,4,2
Tiffany,4,3,4
Tom,1,2,5
Varada,5,2,4


You’ll notice that Pandas set our specified index as the index of the new dataframe and preserved the label of the columns. We can easily remove these names and reset the index to make our dataframe look like it originally did.

In [50]:
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None
df_pivot

,Name,2018,2019,2020
0,Joel,3,1,3
1,Mike,3,4,2
2,Tiffany,4,3,4
3,Tom,1,2,5
4,Varada,5,2,4


# Pivot Table

In [51]:
df = pd.DataFrame({"Name": ["Tom", "Tom", "Mike", "Mike"],
                   "Department": ["CS", "STATS", "CS", "STATS"],
                   "2018": [1, 2, 3, 1],
                   "2019": [2, 3, 4, 2],
                   "2020": [5, 1, 2, 2]}).melt(id_vars=["Name", "Department"], var_name="Year", value_name="Courses")
df

,Name,Department,Year,Courses
0,Tom,CS,2018,1
1,Tom,STATS,2018,2
2,Mike,CS,2018,3
3,Mike,STATS,2018,1
4,Tom,CS,2019,2
5,Tom,STATS,2019,3
6,Mike,CS,2019,4
7,Mike,STATS,2019,2
8,Tom,CS,2020,5
9,Tom,STATS,2020,1


In [52]:
df.pivot(index="Name",
         columns="Year",
         values="Courses")

ValueError: Index contains duplicate entries, cannot reshape

In [53]:
df.pivot_table(index="Name", 
               columns='Year', 
               values='Courses', 
               aggfunc='sum')

Year,2018,2019,2020
Name,,,
Mike,4,6,4
Tom,3,5,6


In [54]:
df.pivot_table(
    index=["Name", "Department"], 
    columns='Year', 
    values='Courses')

Year             2018  2019  2020
Name Department                  
Mike CS           3.0   4.0   2.0
     STATS        1.0   2.0   2.0
Tom  CS           1.0   2.0   5.0
     STATS        2.0   3.0   1.0

# Cross Tab

In [60]:
from io import StringIO

In [67]:
data = """Sample Nationality Handedness
.....: 1 USA Right-handed
.....: 2 Japan Left-handed
.....: 3 USA Right-handed
.....: 4 Japan Right-handed
.....: 5 Japan Left-handed
.....: 6 Japan Right-handed
.....: 7 USA Right-handed
.....: 8 USA Left-handed
.....: 9 Japan Right-handed
.....: 10 USA Right-handed"""

In [68]:
data = pd.read_table(StringIO(data), sep="\s+")
data

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\prana\AppData\Local\Temp\ipykernel_22024\258721106.py:1: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_table(StringIO(data), sep="\s+")


,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [69]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10
